# Encoding a neural network generated with Julia into a JSON file

- Step 1: generate a non-linear dataset
- Step 2: Write data to a CSV file
- Step 3: Build the neural network
- Step 4: Train the neural network
- Step 5: Encode NN to JSON file

In [1]:
# imports
using Pkg; Pkg.activate(".")

using Flux, Plots, Random, Statistics, LaplaceRedux
using Flux.Optimise: update!, Adam
theme(:lime)

using CSV
using DataFrames

using JSON
using Serialization
using Tullio

using LinearAlgebra
using Zygote

Random.seed!(42)

### Step 1: generate a non-linear dataset

In [2]:
xs, ys = LaplaceRedux.Data.toy_data_non_linear(200)
X = hcat(xs...) # bring into tabular format
data = zip(xs,ys)

zip([[1.2810855910752834, 0.9537556211870458], [1.9474772862101295, 1.9447957837448069], [1.6636346147891956, 4.8628177740880245], [1.2489738983854946, 0.5580423579013771], [2.871567531990057, 3.8997622868544735], [2.6735996163580333, 2.060337078130556], [2.2579852508892126, 2.118623127686306], [4.112431480759879, 1.6708445365124773], [4.913409450634119, 4.619188930199143], [0.9249415185713727, 4.583347540628576]  …  [-3.8718411489872095, 4.605441383957622], [-2.82690755957905, 0.9430727186165219], [-2.9682923550013265, 2.782447703989634], [-3.742254048942101, 2.0556485379415905], [-3.0160641261110595, 1.4538519714978864], [-0.8216229870174887, 3.9991152958403244], [-0.676817480886136, 2.6732572835074833], [-1.0647606740819762, 3.84544106009164], [-4.697040240569821, 1.0950690324757328], [-2.521217278628995, 1.1608140597795131]], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

### Step 2: Write data to a CSV file

In [3]:
# Open the CSV file for writing
file = "data.csv"
csv_file = open(file, "w")

# Write the header
write(csv_file, "x1,x2,y\n")

# Write the data
for ((x1, x2), y) in data
    write(csv_file, "$x1,$x2,$y\n")
end

# Close the CSV file
close(csv_file)

### Step 3: Build the neural network

In [4]:
n_hidden = 7
D = size(X,1)
nn = Chain(
    Dense(D, n_hidden, σ),
    Dense(n_hidden, 1)
)
loss(x, y) = Flux.Losses.logitbinarycrossentropy(nn(x), y) 

loss (generic function with 1 method)

### Step 4: Training the neural network

In [5]:
opt = Adam(1e-3)
epochs = 100
avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
show_every = epochs/10

for epoch = 1:epochs
  for d in data
    gs = gradient(Flux.params(nn)) do
      l = loss(d...)
    end
    update!(opt, Flux.params(nn), gs)
  end
  if epoch % show_every == 0
    println("Epoch " * string(epoch))
    @show avg_loss(data)
  end
end

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(2 => 7, σ)
│   summary(x) = 2-element Vector{Float64}
└ @ Flux C:\Users\adeli\.julia\packages\Flux\FWgS0\src\layers\stateless.jl:50


Epoch 10


avg_loss(data) = 0.6876893395930529
Epoch 20


avg_loss(data) = 0.6438441320508719


Epoch 30
avg_loss(data) = 0.5680330287199468

...

Epoch 60
avg_loss(data) = 0.2882214645668864


Epoch 70
avg_loss(data) = 0.22462922818958758


Epoch 80
avg_loss(data) = 0.17650087805464865


Epoch 90
avg_loss(data) = 0.1410102225281298


Epoch 100
avg_loss(data) = 0.11529439479578286


### Step 5: Encode NN to JSON file

In [8]:
serialize_json_nn(nn::Chain)::String = JSON.json([Dict(:weight => nn.layers[i].weight, :bias => nn.layers[i].bias) for i in range(1, length(nn.layers))])
# Export as JSON
write("nn.json", serialize_json_nn(nn))
serialize("nn-binary.jlb", nn)

In [23]:
# Read the JSON file
json_string = read("nn.json", String)
parsed_data = JSON.parse(json_string)

# Convert JSON string back to neural network
nndes = Chain(map(layer -> Dense(convert(Matrix{Float64}, layer["weight"]), convert(Array{Float64, 1}, layer["bias"])), parsed_data))

println(nn)
println(nndes)

MethodError: MethodError: no method matching Matrix{Float64}(::Vector{Any})
Closest candidates are:
  Array{T, N}(::AbstractArray{S, N}) where {T, N, S} at array.jl:626
  Array{T, N}(!Matched::Union{Union{NNlib.BatchedAdjoint{T, <:CUDA.CuArray{T}}, NNlib.BatchedTranspose{T, <:CUDA.CuArray{T}}} where T, Union{Base.LogicalIndex{T, <:Union{NNlib.BatchedAdjoint{T, <:CUDA.CuArray{T}}, NNlib.BatchedTranspose{T, <:CUDA.CuArray{T}}}}, Base.ReinterpretArray{T, N, <:Any, <:Union{SubArray{<:Any, <:Any, var"#s14"}, var"#s14"}} where var"#s14"<:Union{NNlib.BatchedAdjoint{T, <:CUDA.CuArray{T}}, NNlib.BatchedTranspose{T, <:CUDA.CuArray{T}}}, Base.ReshapedArray{T, N, <:Union{Base.ReinterpretArray{<:Any, <:Any, <:Any, <:Union{SubArray{<:Any, <:Any, var"#s15"}, var"#s15"}}, SubArray{<:Any, <:Any, var"#s15"}, var"#s15"}} where var"#s15"<:Union{NNlib.BatchedAdjoint{T, <:CUDA.CuArray{T}}, NNlib.BatchedTranspose{T, <:CUDA.CuArray{T}}}, SubArray{T, N, <:Union{Base.ReinterpretArray{<:Any, <:Any, <:Any, <:Unio